In [12]:
from pyforest import*
lazy_imports()

['from sklearn import svm',
 'import lightgbm as lgb',
 'import spacy',
 'import altair as alt',
 'import keras',
 'import tensorflow as tf',
 'import dash',
 'import bokeh',
 'from sklearn.ensemble import RandomForestRegressor',
 'import pydot',
 'import awswrangler as wr',
 'from sklearn.ensemble import GradientBoostingClassifier',
 'import gensim',
 'import datetime as dt',
 'import sys',
 'import numpy as np',
 'import matplotlib as mpl',
 'from pyspark import SparkContext',
 'import tqdm',
 'import plotly as py',
 'import re',
 'from sklearn.ensemble import GradientBoostingRegressor',
 'from openpyxl import load_workbook',
 'from sklearn.ensemble import RandomForestClassifier',
 'import pickle',
 'import glob',
 'import os',
 'import matplotlib.pyplot as plt',
 'from sklearn.feature_extraction.text import TfidfVectorizer',
 'import plotly.graph_objs as go',
 'from sklearn.model_selection import train_test_split',
 'from sklearn.preprocessing import OneHotEncoder',
 'import nltk',


In [13]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")



In [14]:
Train_data=pd.read_csv('Train.csv')
Test_data=pd.read_csv('Test.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
Train_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [16]:
Test_data.head()


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [19]:
print("Train_data:",Train_data.shape)
print("Test_data:",Test_data.shape)


Train_data: (8523, 12)
Test_data: (5681, 11)


In [21]:
Train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Item_Weight,7060.0,12.857645,4.643456,4.555,8.773750,12.600000,16.850000,21.350000
Item_Visibility,8523.0,0.066132,0.051598,0.000,0.026989,0.053931,0.094585,0.328391
Item_MRP,8523.0,140.992782,62.275067,31.290,93.826500,143.012800,185.643700,266.888400
Outlet_Establishment_Year,8523.0,1997.831867,8.371760,1985.000,1987.000000,1999.000000,2004.000000,2009.000000
Item_Outlet_Sales,8523.0,2181.288914,1706.499616,33.290,834.247400,1794.331000,3101.296400,13086.964800


In [22]:
Train_data.isnull().sum()


Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64